# From here authorship attribution is done

In [ ]:
import pandas as pd
import numpy as np
import nltk
import math
import json
import matplotlib.pyplot as plt
import pycountry
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import collections as coll
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from collections import Counter


analyser = SentimentIntensityAnalyzer()

sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

nltk.download('cmudict')
nltk.download('stopwords')

cmuDictionary = None

df = pd.read_csv('winemag-data-130k-v2.csv')

value_counts = df['taster_name'].value_counts()
to_remove = value_counts[value_counts >= 10000].index
new_dataset = df[~df.taster_name.isin(to_remove)]
voss_only = df[df.taster_name =="Roger Voss"]
schachner_only = df[df.taster_name =="Michael Schachner"]
keefe_only = df[df.taster_name =="Kerin O’Keefe"]

voss_new = voss_only.drop(np.random.choice(voss_only.index, 15514, replace=False))
schachner_new = schachner_only.drop(np.random.choice(schachner_only.index, 5134, replace=False))
keefe_new = keefe_only.drop(np.random.choice(keefe_only.index, 776, replace=False))

new_dataset = pd.concat([new_dataset, keefe_new], axis=0)
new_dataset = pd.concat([new_dataset, schachner_new], axis=0)
new_dataset = pd.concat([new_dataset, voss_new], axis=0)

new_dataset = new_dataset[new_dataset.taster_name != 'Christina Pickard']
new_dataset = new_dataset[new_dataset.taster_name != 'Fiona Adams']
new_dataset = new_dataset[new_dataset.taster_name != 'Carrie Dykes']
new_dataset = new_dataset[new_dataset.taster_name != 'Alexander Peartree']
new_dataset = new_dataset[new_dataset.taster_name != 'Jeff Jenssen']
new_dataset = new_dataset[new_dataset.taster_name != 'Mike DeSimone']
new_dataset = new_dataset[new_dataset.taster_name != 'Susan Kostrzewa']
new_dataset = new_dataset[new_dataset.taster_name != 'Lauren Buzzeo']
new_dataset = new_dataset[new_dataset.taster_twitter_handle != '@AnneInVino']

df_new = new_dataset
print(df_new['taster_name'].value_counts())

In [ ]:
df_new = df_new[['description', 'taster_name']]

df_new = df_new.drop_duplicates()

In [ ]:
from collections import Counter
import re
 
cnt = Counter()
for word in df_new.taster_name:
    cnt[word] += 1
cnt

Counter(df_new.taster_name).most_common(5)

In [ ]:
#check for null values in the authors column
null_values = df_new['taster_name'].isnull().sum()
null_values

#check null values for description
null_values_desc = df_new['description'].isnull().sum()
null_values_desc

#remove authors that are not known
df_new = df_new[df_new['taster_name'].notna()]
df_new

In [ ]:
plot = df_new.groupby([df_new.taster_name]).count().plot(kind='bar')
plot.set_xlabel("cuisines")
plot.set_ylabel("count")

In [ ]:
texts = df_new['description']
authors = df_new['taster_name']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(texts)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectors, authors, test_size=0.2)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
svm = LinearSVC()
svm.fit(X_train, y_train)

In [ ]:
predictions = svm.predict(X_test)
print(list(predictions[0:10]))

In [ ]:
print(y_test[:10])
print(X_train[0:10])

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(predictions,y_test))

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

accuracy= []
writers = [ 'Anna Lee C. Iijima', 'Jim Gordon', 'Joe Czerwinski', "Kerin O'Keefe", 'Matt Kettman', 'Michael Schachner', 'Paul Gregutt', 'Roger Voss', 'Sean P. Sullivan', 'Virginie Boone']

confusion=pd.DataFrame(confusion_matrix(y_test, predictions),columns=writers,index=writers)
plt.figure(figsize=(15,10))
sns.heatmap(confusion,annot=True,fmt='d')
plt.show()

In [ ]:
def syllable_count(word):
    global cmuDictionary
    d = cmuDictionary
    try:
        syl = [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except:
        syl = syllable_count_Manual(word)
    return syl

In [ ]:
def syllable_count_Manual(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("e"):
                count -= 1
    if count == 0:
        count += 1
    return count

In [ ]:
def RemoveSpecialCHs(text):
    text = word_tokenize(text)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

    words = [word for word in text if word not in st]
    return words

In [ ]:
# returns avg number of words in a sentence
def Avg_SentLengthByWord(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])

In [ ]:
# COUNTS SPECIAL CHARACTERS NORMALIZED OVER LENGTH OF CHUNK
def CountSpecialCharacter(text):
    st = ["#", "$", "%", "&", "(", ")", "*", "+", "-", "/", "<", "=", '>',
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return count

In [ ]:
def CountPunctuation(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return float(count)

In [ ]:
def Avg_Syllable_per_Word(text):
    tokens = word_tokenize(text, language='english')
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    syllabls = [syllable_count(word) for word in words]
    p = (" ".join(words))
    return sum(syllabls) / max(1, len(words))

In [ ]:
# K  10,000 * (M - N) / N**2
# , where M  Sigma i**2 * Vi.
def YulesCharacteristicK(text):
    words = RemoveSpecialCHs(text)
    N = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    vi = coll.Counter()
    vi.update(freqs.values())
    M = sum([(value * value) * vi[value] for key, value in freqs.items()])
    K = 10000 * (M - N) / math.pow(N, 2)
    return K

In [ ]:
# returns avg number of characters in a sentence
def Avg_SentLengthByCh(text):
    tokens = sent_tokenize(text)
    return np.average([len(token) for token in tokens])

In [ ]:
def SimpsonsIndex(text):
    words = RemoveSpecialCHs(text)
    freqs = coll.Counter()
    freqs.update(words)
    N = len(words)
    n = sum([1.0 * i * (i - 1) for i in freqs.values()])
    D = 1 - (n / (N * (N - 1)))
    return D

In [ ]:
def number_nn(text): 
    sentences_nltk = sent_tokenize(text)
    for sentence_nltk in sentences_nltk:
        tokens_per_sentence = []
        sent_tokens = word_tokenize(sentence_nltk)
        tokens_per_sentence.append(sent_tokens)
        for token in tokens_per_sentence:
            pos_tags_per_sentence = []
            pos_tags_per_sentence.append(nltk.pos_tag(token))
            word_list = []
            pos_list = []
            for element in pos_tags_per_sentence:
                for otherelement in element:
                    word_list.append(otherelement[0])
                    pos_list.append(otherelement[1])
            number = pos_list.count('NN')
    return number

In [ ]:
import json
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import vader
import spacy
#nlp = spacy.load('en') # en_core_web_sm
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt
import pycountry
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

from textblob import TextBlob

In [ ]:
def TextB(text):
    testimonial = TextBlob(text)
    score = testimonial.sentiment.polarity
    return score

In [ ]:
def VaD(text):
    vader_model = SentimentIntensityAnalyzer() 
    score = vader_model.polarity_scores(text)
    scoore = score['compound']
    return scoore

In [ ]:
SLBY = []
CSC = []
CP = []
ASpW = []
YCK = []
ASLBC = []
SI = []
VD = []
TB = []
NN = []

for text in texts:
    SLBY.append(Avg_SentLengthByWord(text))
    CSC.append(CountSpecialCharacter(text))
    CP.append(CountPunctuation(text))
    ASpW.append(Avg_Syllable_per_Word(text))
    YCK.append(YulesCharacteristicK(text))
    ASLBC.append(Avg_SentLengthByCh(text))
    SI.append(SimpsonsIndex(text))
    VD.append(VaD(text))
    TB.append(TextB(text))
    NN.append(number_nn(text))

In [ ]:
df_new['Average word length'] = SLBY
df_new['Special characters'] = CSC
df_new['Punctuation'] = CP
df_new['Average syllable per word'] = ASpW
df_new['Yules'] = YCK
df_new['Average characters in a sentence'] = ASLBC
df_new["Simpson's Index"] = SI
df_new['VADER'] = VD
df_new['Textblob'] = TB
df_new['Nouns'] = NN
authors = df_new['taster_name']

In [ ]:
df_new = df_new.drop(columns=['description'], axis=1)
df_new = df_new.drop(columns=['taster_name'], axis=1)

In [ ]:
df_new

In [ ]:
features = df_new
features

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB, BernoulliNB

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels = encoder.fit_transform(authors)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf = DecisionTreeClassifier(random_state=0, max_depth=50)
RT = RandomForestClassifier(n_estimators=10)

RT.fit(X_train, y_train)

In [ ]:
predictions = RT.predict(X_test)

In [ ]:
print(predictions[0:100])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(accuracy_score(y_test, predictions))

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")

In [ ]:
dummy_clf.fit(X_train, y_train)

In [ ]:
dummy_clf.score(predictions, y_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

writers = [ 'Anna Lee C. Iijima', 'Jim Gordon', 'Joe Czerwinski', "Kerin O'Keefe", 'Matt Kettman', 'Michael Schachner', 'Paul Gregutt', 'Roger Voss', 'Sean P. Sullivan', 'Virginie Boone']

confusion=pd.DataFrame(confusion_matrix(y_test, predictions),columns=writers,index=writers)
plt.figure(figsize=(15,10))
sns.heatmap(confusion,annot=True,fmt='d')
plt.title("Confusion matrix with all features on Random ")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('confusion.png')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

RT = RandomForestClassifier(n_estimators=10)

RT.fit(X_train, y_train)
predictions = RT.predict(X_test)
print(accuracy_score(y_test, predictions))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf_DT = DecisionTreeClassifier(random_state=0, max_depth=50)

clf_DT.fit(X_train, y_train)

y_pred_DT = clf_DT.predict(X_test)
print(accuracy_score(y_test, y_pred_DT))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf_DT_10 = DecisionTreeClassifier(random_state=0, max_depth=10)

clf_DT_10.fit(X_train, y_train)

y_pred_DT_10 = clf_DT_10.predict(X_test)
print(accuracy_score(y_test, y_pred_DT_10))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf_DT_5 = DecisionTreeClassifier(random_state=0, max_depth=5)

clf_DT_5.fit(X_train, y_train)

y_pred_DT_5 = clf_DT_5.predict(X_test)
print(accuracy_score(y_test, y_pred_DT_5))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf_DT_25 = DecisionTreeClassifier(random_state=0, max_depth=25)

clf_DT_25.fit(X_train, y_train)

y_pred_DT_25 = clf_DT_25.predict(X_test)
print(accuracy_score(y_test, y_pred_DT_25))

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import BernoulliNB
clf_NB = BernoulliNB().fit(X_train, y_train)
y_pred_NB = clf_NB.predict(X_test)
print(accuracy_score(y_test, y_pred_NB))

# From here Sentiment Analysis is done

In [ ]:
import json
import pandas as pd
import numpy as np

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import vader
import spacy
nlp = spacy.load('en') # en_core_web_sm
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt
import pycountry
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

from textblob import TextBlob
df = pd.read_json("wine.json")

In [ ]:
compound = []
vader_model = SentimentIntensityAnalyzer()
for description in df.description: 
    scores = vader_model.polarity_scores(description)
    compound.append(scores['compound'])
points_vader = df.points
compound_vader = compound

df_compound = pd.DataFrame(compound_vader, columns = ['compound'])
df_points = pd.DataFrame(points_vader, columns = ['points'])
from sklearn.linear_model import LinearRegression
horizontal_stack = pd.concat([df_compound, df_points], axis = 1)

X = horizontal_stack.iloc[:, 0].values.reshape(-1, 1)
Y = horizontal_stack.values[:, 1]
lr = LinearRegression()
lr.fit(X, Y)
Y_pred = lr.predict(X)
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.title('Correlation between compound and points')
plt.ylabel('points')
plt.xlabel('compound')
plt.show()
display(horizontal_stack.corr())

In [ ]:
from sklearn import preprocessing
import seaborn as sns
data_df = pd.DataFrame({"compound": compound_vader, "point": points_vader})
scaler = preprocessing.StandardScaler()
# Fit our data on the scaler object
scaled_df = scaler.fit_transform(data_df)
scaled_df = pd.DataFrame(scaled_df, columns=[data_df.columns])
ax = scaled_df.plot(alpha=15)
ax.legend(["compound", "point"]);
plt.ylim(-6, 6)

In [ ]:
compound = []
for description in df.description:
    testimonial = TextBlob(description)
    scores = testimonial.sentiment.polarity
    compound.append(scores)

points_blob = df.points
compound_blob = compound

df_compound = pd.DataFrame(compound_blob, columns = ['compound'])
df_points = pd.DataFrame(points_blob, columns = ['points'])
from sklearn.linear_model import LinearRegression
horizontal_stack = pd.concat([df_compound, df_points], axis = 1)
horizontal_stack

X1 = horizontal_stack.iloc[:, 0].values.reshape(-1, 1)
Y1 = horizontal_stack.values[:, 1]
lr = LinearRegression()
lr.fit(X1, Y1)
Y_pred1 = lr.predict(X1)
plt.scatter(X1, Y1)
plt.plot(X1, Y_pred1, color='red')
plt.title('Correlation between compound and points')
plt.ylabel('points')
plt.xlabel('compound')
plt.show()
display(horizontal_stack.corr())

In [ ]:
from sklearn import preprocessing
import seaborn as sns
data_df = pd.DataFrame({"compound": compound_blob, "point": points_blob})
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(data_df)
scaled_df = pd.DataFrame(scaled_df, columns=[data_df.columns])
ax = scaled_df.plot(alpha=15)
ax.legend(["compound", "point"]);
plt.ylim(-6, 6)

In [ ]:
#taster with the highest mean points 
taster_points = pd.DataFrame({"points": df.points, "taster": df.taster_name})
for taster in taster_points: 
    tasters = taster_points.groupby(['taster'])['points'].mean()
tasters.nlargest(5)


In [ ]:
#taster with highest mean compound text blob 
taster_compound_blob = pd.DataFrame({"compound": compound_blob, "taster": df.taster_name})
for taster in taster_compound_blob: 
    tasters_blob = taster_compound_blob.groupby(['taster'])['compound'].mean()
tasters_blob.nlargest(5)

In [ ]:
#taster with highest mean compound vader
taster_compound_vader = pd.DataFrame({"compound": compound_vader, "taster": df.taster_name})
for taster in taster_compound_vader: 
    tasters_vader = taster_compound_vader.groupby(['taster'])['compound'].mean()
tasters_vader.nlargest(5)

In [ ]:
plot = df.groupby([df.taster_name]).count()["description"].plot(kind="bar")
plot.set_xlabel("taster name")
plot.set_ylabel("count")